# Notebook to experiment with MongoDB

In this notebook you can play around with MongoDB through python.

First we install some needed libraries.

In [ ]:
!pip install pymongo
!pip install folium

Next we load the libraries we are going to use. Most important is pymongo which will interface python with the MongoDB running on the Jupyter server.

In [ ]:
import os
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo
from pymongo import MongoClient
print ('Mongo version', pymongo.__version__)
host = "localhost"
port = 27017

user_name = "test"
pass_word = "mongo"  

db_name = "test"  # database name to authenticate

client = MongoClient(f'mongodb://{user_name}:{pass_word}@{host}:{port}/{db_name}') 
db = client.test
collection = db.people

In [ ]:
collection.drop()
os.system('mongoimport -u test -p "mongo" -d test -c people nosql/dummyData.json')

In [ ]:
cursor = collection.find().sort('Age',pymongo.ASCENDING).limit(3)
for doc in cursor:
    print (doc)

In [ ]:
pipeline = [
        {"$group": {"_id":"$Country",
             "AvgAge":{"$avg":"$Age"},
             "Count":{"$sum":1},
        }},
        {"$sort":{"Count":-1,"AvgAge":1}}
]
aggResult = collection.aggregate(pipeline) # returns a cursor
df1 = pd.DataFrame(list(aggResult)) # use list to turn the cursor to an array of documents
df1 = df1.set_index("_id")
df1.head()

In [ ]:
pipeline = [
        {"$match": {"Country":"China"}},
]
aggResult = collection.aggregate(pipeline)
df2 = pd.DataFrame(list(aggResult))
df2.head()

In [ ]:
import folium
print ('Folium version', folium.__version__)

world_map = folium.Map(location=[35, 100], 
                    zoom_start=4)
for i in range(len(df2)):
    #world_map.simple_marker(location=df2.Location[i].split(','), popup=df2.Name[i]+', age:'+str(df2.Age[i]))
    folium.Marker(df2.Location[i].split(','), popup=df2.Name[i]+', age:'+str(df2.Age[i]),tooltip=df2.Name[i]).add_to(world_map)
    #folium.Marker(
    #[45.3288, -121.6625], popup="<i>Mt. Hood Meadows</i>", tooltip=tooltip
#).add_to(m)
    
world_map